In [2]:
# General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from random import randrange

In [ ]:
# dictionaries with the structure source -> array of followings
dictionary = defaultdict(list)
# simple datafram to store the sources and the count of source followings
data = pd.DataFrame([])

file = open('data/train.txt', 'r') 
lines = file.readlines() 
count = 0
for line in lines: 
    count = count + 1
    split_string = list(map(int,line.strip().split("\t")))
    dictionary[split_string[0]] = []
    if (len(split_string) > 1):
        dictionary[split_string[0]] = sorted(split_string[1:len(split_string)])
        
    data = data.append(pd.DataFrame({'Source': split_string[0], 'Source_Followings': len(dictionary[split_string[0]]) }, index=[0]), ignore_index=True)

In [ ]:
def followingInCommon(node1,node2):  
    list1 = dictionary[node1]
    list2 = dictionary[node2]  
    common_elements = set(list1).intersection(list2) 
    return len(common_elements)
    
# Test    
followingInCommon(750901,3714465) 

In [ ]:
def indirectFollowings(node_source,node_sink):
    count = 0
    list1 = dictionary[node_source]
    for node in list1:
        if node_sink in dictionary[node]:
            count += 1
    return count
    
# Test    
indirectFollowings(750901,3714465) 

In [ ]:
# function to calculate followers
def calculateFollowers(node):
    followers = 0
    for j in range(0,19999):
        if node in dictionary[data.at[j,'Source']]:
            followers = followers + 1
    
    return followers

# Test    
calculateFollowers(161276) 

In [ ]:
# function to calculate followers of the source that are followers of the sink
def SourceFollowersToSink(source,sink):
    count = 0
    for j in range(0,19999):
        node = data.at[j,'Source']
        #if it is a follower of the source
        if source in dictionary[node]:
            #check it is a follower of the sink
            if sink in dictionary[node]:
                count +=  1
    
    return count

# Test    
SourceFollowersToSink(750901,3714465) 

In [17]:
fake_edges4 = pd.read_csv("data_generated/fake_edges4.csv", sep='\t')
real_edges3 = pd.read_csv("data_generated/real_edges3.csv", sep='\t')
test_data = pd.read_csv("data_generated/test_data.csv", sep='\t')

In [ ]:
real_edges3['Real'] = 1
fake_edges4['Real'] = 0

In [ ]:
real_edges3['Common_Followings'] = real_edges3.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
real_edges3['Com_Followings_Ratio'] = real_edges3['Common_Followings']/real_edges3['Source_Followings']
real_edges3['Followers_Ratio'] = real_edges3['Sink_Followers']/real_edges3['Source_Followers']
real_edges3['Indirect_Followings'] = real_edges3.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
real_edges3['Inv_Indirect_Followings'] = real_edges3.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)
real_edges3['SourceFollowersToSink'] = real_edges3.apply(lambda x: SourceFollowersToSink(x['Source'], x['Sink']), axis=1)

In [ ]:
fake_edges4['Common_Followings'] = fake_edges4.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
fake_edges4['Com_Followings_Ratio'] = fake_edges4['Common_Followings']/fake_edges4['Source_Followings']
fake_edges4['Followers_Ratio'] = fake_edges4['Sink_Followers']/fake_edges4['Source_Followers']
fake_edges4['Indirect_Followings'] = fake_edges4.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
fake_edges4['Inv_Indirect_Followings'] = fake_edges4.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)
fake_edges4['SourceFollowersToSink'] = fake_edges4.apply(lambda x: SourceFollowersToSink(x['Source'], x['Sink']), axis=1)

In [ ]:
test_data['Common_Followings'] = test_data.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
test_data['Com_Followings_Ratio'] = test_data['Common_Followings']/test_data['Source_Followings']
test_data['Followers_Ratio'] = test_data['Sink_Followers']/test_data['Source_Followers']
test_data['Indirect_Followings'] = test_data.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
test_data['Inv_Indirect_Followings'] = test_data.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)
test_data['SourceFollowersToSink'] = test_data.apply(lambda x: SourceFollowersToSink(x['Source'], x['Sink']), axis=1)

In [ ]:
real_edges3.to_csv("data_generated/real_edges3.csv", sep='\t', index=False)
fake_edges4.to_csv("data_generated/fake_edges4.csv", sep='\t', index=False)
test_data.to_csv("data_generated/test_data.csv", sep='\t', index=False)

In [18]:
frames = [real_edges3,fake_edges4]
dataset = pd.concat(frames, ignore_index=True)

# Saving real edges
dataset.to_csv("data_models/dataset7.csv", sep='\t', index=False)

In [ ]:
def checkSink(node):
    check = 0
    if ( (data['Source'] == node).any() ):
        check = 1         
    return check

End 